# Topic Modeling Assessment Project

Welcome to your Topic Modeling Assessment! For this project you will be working with a dataset of over 400,000 quora questions that have no labeled cateogry, and attempting to find optimal number of cateogries to assign these questions to. The .csv file of these text questions can be found in the NLP folder.


#### Task: Import pandas and read in the quora_questions.csv file.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('quora_questions.csv')

In [3]:
df.head()

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


In [16]:
df.tail()

,Question
404284,How many keywords are there in the Racket prog...
404285,Do you believe there is life after death?
404286,What is one coin?
404287,What is the approx annual cost of living while...
404288,What is like to have sex with cousin?


# Preprocessing

#### Task: Create a vectorized document term matrix. 

- How do you want to clean up your text with regards to stopwords, special characters, and other situations.
- Using a Countvectorizer versus a TFIDFvectorizer
- You may want to explore the max_df and min_df parameters. 


In [4]:
import re
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [5]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(text):
    # remove html tags from all of the text before processing
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext, disable=['parser', 'ner'])

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=10, stop_words='english', use_idf=True)

In [8]:
dtm = tfidf.fit_transform(df['Question'])

In [9]:
dtm

<404289x11984 sparse matrix of type '<class 'numpy.float64'>'
	with 1838887 stored elements in Compressed Sparse Row format>

# LDA Modelling

#### TASK: Using Scikit-Learn create an instance of LDA. 

- You can manually run and tune your model, then evaluate the resulting clusters. 
- Or you can use gridsearch to try and identify the best number of topics to use. 


In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [20]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,           # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=20,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [24]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


array([[0.01527087, 0.01527087, 0.01527087, ..., 0.01527087, 0.01527087,
        0.01527087],
       [0.0168838 , 0.0168838 , 0.0168838 , ..., 0.0168838 , 0.0168838 ,
        0.0168838 ],
       [0.01471685, 0.01471685, 0.1369912 , ..., 0.01471685, 0.01471685,
        0.01471685],
       ...,
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.12348292, 0.07774096, 0.01332946, ..., 0.09184247, 0.01332946,
        0.01332946],
       [0.01886073, 0.01886073, 0.01886073, ..., 0.01886073, 0.01886073,
        0.01886073]])

#### Task: Evaluate the different models you have run and determine which model you think determines the best clusters.  


The evaluation part could invlove:
- Printing out the top 15 most common words for each of the topics and seeing if they make sense.
- Using the perplexity and log-likelihoood scores.
- Using the pyLDAvis tool to investigate the different clusters. 

In [25]:
# Log Likelihood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -8075983.515450307
Perplexity:  20842.46240486253
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [26]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['mobile', 'food', 'china', 'right', 'experience', 'relationship', 'eat', 'android', 'app', 'live', 'phone', 'friend', 'like', 'account', 'think']


THE TOP 15 WORDS FOR TOPIC #1
['research', 'bollywood', 'gift', '5', 'mba', 'product', 'form', 'apply', 'fact', 'interesting', 'date', 'earth', 'big', 'indian', 'example']


THE TOP 15 WORDS FOR TOPIC #2
['average', 'test', 'major', '10', 'rid', '2017', 'look', 'meaning', 'university', 'good', 'new', 'online', 'money', 'way', 'life']


THE TOP 15 WORDS FOR TOPIC #3
['female', 'uber', 'marry', 'save', 'matter', 'advice', 'film', 'value', 'break', 'remove', 'period', 'c', 'woman', 'human', 'happen']


THE TOP 15 WORDS FOR TOPIC #4
['night', 'color', 'hear', 'cell', 'overcome', 'kill', 'light', 'function', 'dream', 'pay', 'favorite', 'travel', 'hack', 'bad', 'work']


THE TOP 15 WORDS FOR TOPIC #5
['good', 'future', 'age', 'end', 'speak', 'girlfriend', 'child', 'great', 'hillary', 'president', 'clinton', 'come', 

In [50]:
# original output

THE TOP 15 WORDS FOR TOPIC #0
['thing', 'read', 'place', 'visit', 'places', 'phone', 'buy', 'laptop', 'movie', 'ways', '2016', 'books', 'book', 'movies', 'best']


THE TOP 15 WORDS FOR TOPIC #1
['majors', 'recruit', 'sex', 'looking', 'differ', 'use', 'exist', 'really', 'compare', 'cost', 'long', 'feel', 'work', 'mean', 'does']


THE TOP 15 WORDS FOR TOPIC #2
['add', 'answered', 'needing', 'post', 'easily', 'improvement', 'delete', 'asked', 'google', 'answers', 'answer', 'ask', 'question', 'questions', 'quora']


THE TOP 15 WORDS FOR TOPIC #3
['using', 'website', 'investment', 'friends', 'black', 'internet', 'free', 'home', 'easy', 'youtube', 'ways', 'earn', 'online', 'make', 'money']


THE TOP 15 WORDS FOR TOPIC #4
['balance', 'earth', 'day', 'death', 'changed', 'live', 'want', 'change', 'moment', 'real', 'important', 'thing', 'meaning', 'purpose', 'life']


THE TOP 15 WORDS FOR TOPIC #5
['reservation', 'engineering', 'minister', 'president', 'company', 'china', 'business', 'country', 

In [27]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [15, 20, 25],'learning_decay': [.5, .7]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)
# Create Document - Topic Matrix
lda_output = lda_model.transform(dtm)

# column names
# best_lda_model = model.best_estimator_
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [18]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, dtm, vectorizer, mds='tsne')
panel

/Users/stephaniekendall/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


NameError: name 'best_lda_model' is not defined

#### TASK: Add a new column to the original quora dataframe that labels each question into one of the topic categories.

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


,Question,Topic
0,What is the step by step guide to invest in sh...,5
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,16
2,How can I increase the speed of my internet co...,17
3,Why am I mentally very lonely? How can I solve...,11
4,"Which one dissolve in water quikly sugar, salt...",14
5,Astrology: I am a Capricorn Sun Cap moon and c...,1
6,Should I buy tiago?,0
7,How can I be a good geologist?,10
8,When do you use シ instead of し?,19
9,Motorola (company): Can I hack my Charter Moto...,17


# Great job!